In [1]:
from imgaug import augmenters as iaa
from Manager import *
from VisionUtils import *
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
import imgaug as ia
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


In [2]:
tf.enable_eager_execution()

In [39]:
IMG_RT_PATH = 'data/images/'
FEATURE_RT_PATH = "data/processed/Image_features/"
FEATURE_AUG_RT_PATH = "data/image_aug_features/"
IMG_AUG_RT_PATH = 'data/images_aug/'

#img_list = pd.read_csv(IMG_RT_PATH + "test_list.TXT")
feature_extractor = load_model("Models\FaceNet/Facenet_keras.h5")
face_detector = MTCNN()

W0930 21:54:02.987272  9540 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [40]:

seq = iaa.Sequential([iaa.MotionBlur(k = 20)])

In [41]:
os.listdir("data/processed/")[0]

'images'

In [42]:
import os 
img_path_list = []
folder_list = os.listdir(IMG_RT_PATH)
for FOLDER in folder_list:
    image_list_names = os.listdir(IMG_RT_PATH + FOLDER)
    for NAME in image_list_names:
        img_path_list.append(FOLDER+'/'+NAME)

In [43]:
feat_path_list = []
folder_list = os.listdir(FEATURE_RT_PATH)
for FOLDER in folder_list:
    image_list_names = os.listdir(FEATURE_RT_PATH + FOLDER)
    for NAME in image_list_names:
        feat_path_list.append(FOLDER+'/'+NAME)

In [44]:
feat_path_list[0]

'n000002/0001_01.npy'

In [45]:
img_list = pd.DataFrame({"path": feat_path_list})

In [46]:
img_list

,path
0,n000002/0001_01.npy
1,n000002/0002_01.npy
2,n000002/0003_01.npy
3,n000002/0004_01.npy
4,n000002/0005_01.npy
5,n000002/0006_01.npy
6,n000002/0007_01.npy
7,n000002/0008_01.npy
8,n000002/0009_01.npy
9,n000002/0010_01.npy


In [47]:
iman = ImageManager(
    imgpaths = img_list,
    feature_extractor = feature_extractor,
    face_detector =  face_detector, 
    IMG_RTPATH = IMG_RT_PATH,
)

In [48]:
iman.balance_random_sample(30)
paths = iman.get_sample_image_paths()
iman.split(train_size = 0.8)

In [49]:
path1 = np.array(img_list)[iman.sample_feat1]
path2 = np.array(img_list)[iman.sample_feat2]
path1, path2 = shuffle(path1, path2, random_state = 42)

In [50]:
len(path1)*0.75

48465.0

In [51]:
path1[:10]

array([['n000441/0035_02.npy'],
       ['n000182/0044_01.npy'],
       ['n000795/0145_01.npy'],
       ['n000295/0049_01.npy'],
       ['n000683/0162_02.npy'],
       ['n000592/0049_01.npy'],
       ['n000856/0273_01.npy'],
       ['n000461/0010_01.npy'],
       ['n000839/0384_01.npy'],
       ['n000239/0268_01.npy']], dtype=object)

In [52]:
xtrain1 = path1[:int(len(path1)*.75)]
xtrain2 = path2[:int(len(path2)*.75)]
xtest1 = path1[int(len(path1)*.75):]
xtest2 = path2[int(len(path2)*.75):]

In [53]:
def extract_feature2( feature_extractor, img, bb):
    """
    Extract features given img, bounding box and feature extractor
    """
    insz = feature_extractor.input_shape
    img = resize(img,(insz[1],insz[2]))
    img = normalize(img).reshape((1, insz[1],insz[2], insz[3]))
    image_feature = feature_extractor.predict(img)
    return image_feature

In [54]:
i = 0
augment_mode = False
method = "imgaug"
training_features1 = []
training_features2 = []
p2, p1, train_labels = [], [], []

In [55]:
path1 = xtrain1
path2 = xtrain2

In [56]:
len(path1)

48465

In [57]:
path1[i][0].split('/')[0]

'n000441'

In [58]:
FEATURE_RT_PATH + str(path1[i][0])

'data/processed/Image_features/n000441/0035_02.npy'

In [59]:
for i in tqdm(range(len(path1))):
    training_features1.append(np.load(FEATURE_RT_PATH + str(path1[i][0])))
    training_features2.append(np.load(FEATURE_RT_PATH + str(path2[i][0])))
    if(path1[i][0].split('/')[0] ==path2[i][0].split('/')[0]):
        train_labels.append(1)
    else:
        train_labels.append(0)  

100%|██████████████████████████████████████████████████████████████████| 48465/48465 [07:19<00:00, 110.38it/s]


In [60]:
print("Number of features in feat1: {}".format(len(training_features1)))
print("Number of features in feat2: {}".format(len(training_features2)))
print("Number of features in labels: {}".format(len(train_labels)))

Number of features in feat1: 48465
Number of features in feat2: 48465
Number of features in labels: 48465


In [61]:
len(training_features1)

48465

In [62]:
len(train_labels)

48465

In [63]:
path1_t = xtest1
path2_t = xtest2
test_features1, test_features2 = [], []
test_labels = []

In [64]:
for i in tqdm(range(len(path1_t))):
    test_features1.append(np.load(FEATURE_RT_PATH + path1_t[i][0]))
    test_features2.append(np.load(FEATURE_RT_PATH + path2_t[i][0]))
    if(path1_t[i][0].split('/')[0] == path2_t[i][0].split('/')[0]):
        test_labels.append(1)
    else:
        test_labels.append(0)

100%|██████████████████████████████████████████████████████████████████| 16155/16155 [02:01<00:00, 133.16it/s]


In [65]:
train_feat1 = np.vstack(training_features1)
train_feat2 = np.vstack(training_features2)

test_feat1 = np.vstack(test_features1)
test_feat2 = np.vstack(test_features2)

In [66]:
np.save("features/train_feat1.npy", train_feat1) 
np.save("features/train_feat2.npy", train_feat2)
np.save("features/train_labels.npy", train_labels)


In [67]:
np.save("features/test_feat1.npy", test_feat1) 
np.save("features/test_feat2.npy", test_feat2)
np.save("features/test_labels.npy", test_labels)
